# Funciones ETo
***

_Autor_: __Jesús Casado__<br>
_Fecha_: __20/08/2020__<br>

__Introducción__<br>


__Errores a corregir__<br>


__Índice__<br>


In [ ]:
import numpy as np
from datetime import datetime

In [ ]:
def estimar_Δ(Tmed):
    """Pendiente de la curva de presión de vapor de saturación (kPa/°C).
    
    Entrada:
    --------
    Tmed:   float. Temperatura media en el periodo de cálculo
    
    Salida:
    -------
    Δ:      float. Pendiente de la curva (kPa/°C)
    """
    
    Δ = 4098 * 0.6108 * np.exp(17.27 * Tmed / (Tmed + 237.3)) / (Tmed + 237.3)**2
    
    return Δ

In [ ]:
def estimar_Patm(z):
    """Estima la presión atmosférica a la cota dada suponiendo una presión estándar a nivel de mar (101.3 Kpa)
    
    Entrada:
    --------
    z:      float. Elevación sobre el nivel del mar (m)
    
    Salida:
    -------
    P:      float. Presión (kPa)
    """
    
    P = 101.3 * ((293 - 0.0065 * z) / 293)**5.26
    
    return P

In [ ]:
def estimar_ϒ(P):
    """Calcula la constante psicrométrica.
    
    Entrada:
    --------
    
    Salida:
    -------
    ϒ:     float. Constante psicrométrica (kPa/°C)
    """
    
    cp = 1.013e-3 # calor específico a presión constante (MJ/(kg·°C))
    ε = 0.622 # cociente del peso molecular de vapor de agua/aire seco
    λ = 2.45 # calor latente de vaporización (MJ/kg)
    
    ϒ = cp * P / (ε * λ)
    
    return ϒ

In [ ]:
def estimar_PV(T):
    """Presión de vapor de saturación para una temperatura del aire
    
    Entrada:
    --------
    T:      float. Temperatura del aire (°C)
    
    Salida:
    -------
    e:      float. Presión de vapor de saturación (kPa)
    """
    
    e = 0.611 * np.exp(17.27 * T / ( T + 237.3))
    
    return e

In [ ]:
def estimar_Ra(φ, ω, Δt, J, verbose=False):
    """Radiación extraterrestre para periodos horarios o menores (MJ/(m²·h)).
    
    Entradas:
    ---------
    φ:       float. Latitud (rad)
    ω:       float. Ángulo solar en el momento en que ocurre el punto medio del periodo considerado (rad).
    Δt:      float. Duración del periodo considerado (h). P.ej: 0.5 para periodos de 30 min
    
    """
    
    Gse = 0.082 # constante solar (MJ·m-²·min-¹)
    dr = 1 + 0.033 * np.cos(2 * np.pi / 365 * J)
    δ = 0.409 * np.sin(2 * np.pi / 365 * J - 1.39)
    if verbose:
        print('dr = {0:.3f}\tδ = {1:.3f}'.format(dr, δ))
    ω1 = ω - np.pi * Δt / 24
    ω2 = ω + np.pi * Δt / 24
    
    Ra = 12 * 60 / np.pi * Gse * dr * ((ω2 - ω1) * np.sin(φ) * np.sin(δ) + np.cos(φ) * np.cos(δ) * (np.sin(ω2) - np.sin(ω1)))
    
    return Ra

In [ ]:
def anguloSolar(t, J, Lz, Lm):
    """
    
    Entradas:
    ---------
    t:       float. Hora estándar en el punto medio del periodo considerado (h). P.ej. para un periodo entre las 14:00 y las 15:00, t = 14.5
    J:       int. Nº de día del año
    Lz:      float. Longitud del centro de la zona de tiempo local (° oeste de Greenwich)
    Lm:      float. Longitud de la zona de medición (° oeste de Greenwich)
    """
    
    # calcular corrección estacional para el tiempo solar (Sc)
    b = 2 * np.pi * (J - 81) / 364
    Sc = 0.1645 * np.sin(2 * b) - 0.1255 * np.cos(b) - 0.025 * np.sin(b)

    ω = np.pi / 12 * ((t + 0.06667 * (Lz - Lm) + Sc) -12)
    
    return ω

In [ ]:
def estimar_Rs(Ra, Td_min, Td_max, k):
    """Radiación de onda corta (MJ/(m²·h))
    """
    
    Rs = k * np.sqrt((Td_max - Td_min)) * Ra
    
    return Rs

In [ ]:
def estimar_Rso(z, Ra):
    
    Rso = (0.75 + 2e-5 * z) * Ra
    
    return Rso

In [ ]:
def estimar_Rns(Rs, α):
    """
    """
    
    Rns = (1 - α) * Rs
    
    return Rns

In [ ]:
def estimar_Rnl(Tmax, Tmin, ea, Rs, Rso, Δt):
    """Estima la radiación neta de onda larga (MJ·m-2·d-1).
    
    Entradas:
    ---------
    Tmax:    float. Temperatura máxima durante un periodo de 24 h (°C)
    Tmin:    float. Temperatura mínima durante un periodo de 24 h (°C)
    ea:      float. Presión de vapor real (kPa)
    Rs:      float. Radiación solar medida o calculada (MJ·m-2·d-1)
    Rso:     float. Radiación en un día despejado (MJ·m-2·d-1)
    Δt:      float. Duración del paso temporal (h)
    
    Salida:
    -------
    Rnl:     float. Radiación neta de onda larga (MJ·m-2·d-1)
    """
    
    # constante de Stefan-Boltzmann
    nSteps = 24 / Δt
    σ = 4.903e-9 / nSteps # MJ·K-4·m-2·d-1; dividir por el número de pasos en 24 h
    
    # convertir temperaturas a K
    Tmax += 273.16
    Tmin += 273.16
    
    # cociente de radiación sobre un día despejado
    R_ratio = min(Rs / Rso, 1)
    Rnl = σ * (Tmax**4 + Tmin**4) / 2 * (0.34 - 0.14 * np.sqrt(ea)) * (1.35 * R_ratio - 0.35)
    
    return Rnl

In [ ]:
def estimar_G(Rn):
    
    if Rn < 0:
        # durante periodos nocturnos
        Ghr = 0.5 * Rn
    else:
        # durante periodos de luz
        Ghr = 0.1 * Rn
    
    return Ghr

In [ ]:
def estimar_ETo(Tmax, Tmin, lat, lon, z, date, Δt, Lz, Td_min, Td_max, α, k=0.19, u2=None, 
                verbose=False):
    """
    
    Entradas:
    ---------
    Rn:      float. Radiación neta en la superficie de referencia (MJ·m-2·d-1)
    G:       float. Densidad del flujo del calor del suelo (MJ·m-2·d-1)
    Tmed:     float. Temperatura media del aire cada hora (°C)
    Δ:       float. Pendiente de la curva de presión de saturación de vapor en Tmed (kPa °C-1)
    ϒ:       float. Constante psicrométrica (kPa °C-1)
    eo:      float. Presión de saturación de vapor a temperatura del aire Tmed (kPa)
    ea:      float. Promedio horario de la presión real de vapor (kPa)
    u2:      float. Promedio horario de la velocidad del viento (m·s-1)
    
    Salida:
    -------
    Eto:     float. Evapotranspiración de referencia (mm·h-1)"""
    
    
    # Pendiente de la curva de presión de saturación de vapor en Tmed (kPa °C-1)
    Tmed = np.mean((Tmax, Tmin)) # temperatura media en el periodo
    Δ = estimar_Δ(Tmed)
    if verbose:
        print('Δ = {0:.3f}'.format(Δ))

    # Constante psicrométrica (kPa °C-1)
    P = estimar_Patm(z)
    ϒ = estimar_ϒ(P)
    if verbose:
        print('ϒ = {0:.3f}'.format(ϒ))
    
    # ESTIMACIÓN DE LOS DATOS DE HUMEDAD
    
    # Presión de saturación de vapor a temperatura del aire Tmed (kPa)
#     Tdew = Td_min # Temperatura de rocío (ºC)
    ea = estimar_PV(Td_min) # donde Tmin se considera la temperatura de rocío
    if verbose:
        print('ea = {0:.3f}'.format(ea))
    
    # Presión de saturación de vapor a temperatura del aire Tmed (kPa)
    eo_max = estimar_PV(Tmax)
    eo_min = estimar_PV(Tmin)
    es = np.mean((eo_max, eo_min))
    if verbose:
        print('es = {0:.3f}'.format(es))
    
    # humedad relativa (%)
    HRmax = ea / eo_min * 100
    HRmin = ea / eo_max * 100
    
    # ESTIMACIÓN DE LOS DATOS DE RADIACIÓN
    
    # día juliano (-)
#     dt = datetime.strptime(date, '%Y-%m-%d %H:%M')
    J = date.timetuple().tm_yday
    t = date.hour + Δt / 2
    if verbose:
        print('J = {0}\tt = {1}'.format(J, t))
    ω = anguloSolar(t, J, Lz, lon)
    if verbose:
        print('ω = {0:.3f}'.format(ω))
    
    
#     print(φ, ω, Δt)
    φ = lat * np.pi / 180
    Ra = estimar_Ra(φ, ω, Δt, J)
    Rs = estimar_Rs(Ra, Td_min, Td_max, k)
    
    Rso = estimar_Rso(z, Ra)
    
    Rns = Rns = (1 - α) * Rs
    if verbose:
        print('Ra = {0:.3f}\tRs = {1:.3f}\tRso = {2:.3f}\tRns = {3:.3f}'.format(Ra, Rs, Rso, Rns))
    
    # radiación neta de onda larga
    Rnl = estimar_Rnl(Tmax, Tmin, ea, Rs, Rso, Δt)
    if verbose:
        print('Rnl = {0:.3f}'.format(Rnl))
    
    # radiación neta
    Rn = Rns - Rnl
    
    # flujo de energía desde el suelo
    G = estimar_G(Rn)
    if verbose:
        print('G = {0:.3f}'.format(G))
    
    # Promedio horario de la velocidad del viento (m·s-1)
    if u2 is None:
        u2 = 2
    
    ETo = (0.408 * Δ * (Rn - G) + ϒ * 37 / (Tmed + 273) * u2 * (es - ea)) / (Δ + ϒ * (1 + 0.34 * u2))
    ETo = max(0, ETo)
    
    return ETo

In [ ]:
def estimar_ETo_serie(T, lat, lon, z, Δt, Lz, α, k=0.19, u2=None,
                      verbose=False, **kwargs):
    """
    
    Entradas:
    ---------
    Rn:      float. Radiación neta en la superficie de referencia (MJ·m-2·d-1)
    G:       float. Densidad del flujo del calor del suelo (MJ·m-2·d-1)
    Tmed:     float. Temperatura media del aire cada hora (°C)
    Δ:       float. Pendiente de la curva de presión de saturación de vapor en Tmed (kPa °C-1)
    ϒ:       float. Constante psicrométrica (kPa °C-1)
    eo:      float. Presión de saturación de vapor a temperatura del aire Tmed (kPa)
    ea:      float. Promedio horario de la presión real de vapor (kPa)
    u2:      float. Promedio horario de la velocidad del viento (m·s-1)
    
    Salida:
    -------
    Eto:     float. Evapotranspiración de referencia (mm·h-1)"""
    
    # temperatura máxima y mínima en cada periodo
    freq = '{0:.0f}min'.format(Δt * 60)
    Tmin = T.resample(freq).min()
    Tmax = T.resample(freq).max()
    
    # temperatura máxima y mínima en 24 h
    window = int(24/Δt)
    Td_max = T.rolling(window, min_periods=10, center=True).max()[Tmin.index]
    Td_min = T.rolling(window, min_periods=10, center=True).min()[Tmin.index]
    
    # calcular serie de evapotranspiración potencial
    ETo = pd.Series(index=Tmin.index, dtype='float64')
    for date in ETo.index:
        ETo[date] = estimar_ETo(Tmax[date], Tmin[date], lat, lon, z, date, Δt, Lz=0,
                                 Td_max=Td_max[date], Td_min=Td_min[date],
                                 α=α, verbose=False)
    
    # redondear la serie
    decimals = kwargs.get('decimals', 3)
    ETo = ETo.round(decimals=decimals)
    
    return ETo